# Modeling for all studies

In [1]:
# import import_ipynb
# import us_common_functions

# Setting variables

In [2]:
#####################################################################################
run_models = True
overleaf_metrics = False
print_all = False
is_deadline = False
#####################################################################################


#####################################################################################
# ['Random_Forest', 'KNN']
model_name = 'Random_Forest'
#####################################################################################


#####################################################################################
k_fold_n_splits = 5
#####################################################################################



#####################################################################################
# feature_imp = True
# normalized_feature = True
#####################################################################################


#####################################################################################
# arousal_signals = ['PP']
# arousal_signals = ['PP', 'BR']
arousal_signals = ['PP', 'HR', 'BR', 'PP_HR', 'PP_BR', 'HR_BR', 'PP_HR_BR']
#####################################################################################



#####################################################################################
%run us_common_functions.ipynb
#####################################################################################




####################################################################################
#### study_combinations_to_run = list(study_combinations.keys())
#### study_combinations_to_run = list(study_combinations.keys())[:5]
#### study_combinations_to_run = ['sim1', 'sim1___sim2___tt1___office_tasks']
#### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
#### study_combinations_to_run = ['sim1', 'tt1']
#### study_combinations_to_run = ['sim1']
#### study_combinations_to_run.remove(deadline_study)
####################################################################################






####################################################################################
### study_combinations_to_run = all_study_combinations[:-1]
### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']

four_studies = [sim1, sim2, tt1, office_tasks]
study_combinations, all_study_combinations = get_study_combinations(four_studies)
study_combinations_to_run = all_study_combinations
print(study_combinations_to_run)
####################################################################################




# #####################################################################################
# # For Deadline Study - Without BR
# #####################################################################################
# studies.append(deadline_study)
# study_combinations, all_study_combinations = get_study_combinations(studies)
# study_combinations_to_run = studies
# print(study_combinations_to_run)

# arousal_signals = ['PP', 'HR', 'PP_HR']
# model_features.remove('BR_Mean')
# model_features.remove('BR_SD')

# is_deadline=True
# ####################################################################################





####################################################################################
# ground_truth_method = 'hist'
# ground_truth_method = 'ecdf'
####################################################################################





['sim1', 'sim2', 'tt1', 'office_tasks', 'sim1___sim2', 'sim1___sim2___tt1', 'sim1___sim2___office_tasks', 'sim1___sim2___tt1___office_tasks']


In [3]:
####################################################################################
# Initialization
####################################################################################
model_metrics = {}
overleaf_final_tables = defaultdict(lambda : None)
study_combination_model_metrics = defaultdict(lambda : None)

# figure_path = all_studies_fig_dir
# plot_name = None
####################################################################################

In [4]:
# data_3_study_combined.csv
# data_3_study_combined_scaled.csv
# data_3_study_combined_nn.csv
# data_3_study_combined_scaled_nn.csv
all_studies_df = pd.read_csv(all_studies_data_dir + 'data_3_study_combined.csv')
all_studies_df.shape

(256541, 56)

In [5]:
all_studies_df.head()

,Unnamed: 0,Subject,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,Gender,Gender_Male,Perinasal_Mean_NN,Perinasal_Median_NN,Perinasal_SD_NN,Perinasal_SS_NN,Hr_Mean_NN,Hr_Median_NN,Hr_SD_NN,Hr_SS_NN,Br_Mean_NN,Br_Median_NN,Br_SD_NN,Br_SS_NN,Perinasal_Mean,Perinasal_Median,Perinasal_SD,Perinasal_SS,HR_Mean,HR_Median,HR_SD,HR_SS,BR_Mean,BR_Median,BR_SD,BR_SS,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Study_Name,Study_Subject,Treatment,PP_RB_Mean,HR_RB_Mean,BR_RB_Mean,PP_Arousal_Mode_Hist,HR_Arousal_Mode_Hist,BR_Arousal_Mode_Hist,PP_HR_Arousal_Mode_Hist,PP_BR_Arousal_Mode_Hist,HR_BR_Arousal_Mode_Hist,PP_HR_BR_Arousal_Mode_Hist
0,0,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.234157,-5.225352,0.022537,273.968589,90.34,90.3,1.484887,81633.00,11.052,11.05,0.162809,1221.7056,0.020212,0.029017,0.022537,0.008657,3.708182,3.668182,1.484887,157.350124,-5.414682,-5.416682,0.162809,293.426352,neutral,stressed,relaxed,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.310385,-5.314152,0.010595,282.002952,91.20,90.6,1.366260,83191.20,12.004,12.10,0.352742,1442.0800,-0.056016,-0.059782,0.010595,0.032388,4.568182,3.968182,1.366260,225.482851,-4.462682,-4.366682,0.352742,200.275130,relaxed,stressed,relaxed,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.301983,-5.300184,0.007855,281.110844,90.08,90.3,1.347261,81160.40,13.108,13.10,0.355928,1719.3368,-0.047614,-0.045814,0.007855,0.023226,3.448182,3.668182,1.347261,135.235579,-3.358682,-3.366682,0.355928,113.947596,relaxed,stressed,relaxed,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.376241,-5.384108,0.022728,289.044320,86.64,86.8,1.188089,75077.60,14.164,14.08,0.319138,2007.1056,-0.121872,-0.129739,0.022728,0.153176,0.008182,0.168182,1.188089,12.704669,-2.302682,-2.386682,0.319138,53.940076,relaxed,neutral,relaxed,relaxed,relaxed,neutral,relaxed,sim1,sim1_2,RD,-5.254369,86.631818,16.466682,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.412159,-5.415029,0.008453,292.915248,86.70,87.6,1.708150,75195.16,15.488,15.55,0.339830,2399.8208,-0.157789,-0.160660,0.008453,0.249618,0.068182,0.968182,1.708150,26.306488,-0.978682,-0.916682,0.339830,10.617541,relaxed,neutral,neutral,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# if ground_truth_method == 'hist':
    
#     all_studies_df.rename(columns = {
        
#                 'PP_Arousal_Mode': 'PP_Arousal_Mode_ecdf',
#                 'HR_Arousal_Mode': 'HR_Arousal_Mode_ecdf',
#                 'BR_Arousal_Mode': 'BR_Arousal_Mode_ecdf',

#                 'PP_HR_Arousal_Mode': 'PP_HR_Arousal_Mode_ecdf',
#                 'PP_BR_Arousal_Mode': 'PP_BR_Arousal_Mode_ecdf',
#                 'HR_BR_Arousal_Mode': 'HR_BR_Arousal_Mode_ecdf',
#                 'PP_HR_BR_Arousal_Mode': 'PP_HR_BR_Arousal_Mode_ecdf',
        
#                 'PP_Arousal_Mode_Hist': 'PP_Arousal_Mode',
#                 'HR_Arousal_Mode_Hist': 'HR_Arousal_Mode',
#                 'BR_Arousal_Mode_Hist': 'BR_Arousal_Mode',

#                 'PP_HR_Arousal_Mode_Hist': 'PP_HR_Arousal_Mode',
#                 'PP_BR_Arousal_Mode_Hist': 'PP_BR_Arousal_Mode',
#                 'HR_BR_Arousal_Mode_Hist': 'HR_BR_Arousal_Mode',
#                 'PP_HR_BR_Arousal_Mode_Hist': 'PP_HR_BR_Arousal_Mode',

#     #             'PP_Arousal_Mode_ecdf': 'PP_Arousal_Mode',
#     #             'HR_Arousal_Mode_ecdf': 'HR_Arousal_Mode',
#     #             'BR_Arousal_Mode_ecdf': 'BR_Arousal_Mode',

#     #             'PP_HR_Arousal_Mode_ecdf': 'PP_HR_Arousal_Mode',
#     #             'PP_BR_Arousal_Mode_ecdf': 'PP_BR_Arousal_Mode',
#     #             'HR_BR_Arousal_Mode_ecdf': 'HR_BR_Arousal_Mode',
#     #             'PP_HR_BR_Arousal_Mode_ecdf': 'PP_HR_BR_Arousal_Mode',

#             }, inplace=True)

In [7]:
for arousal_signal in arousal_signals:
    print_percentage(all_studies_df, arousal_signal+'_Arousal_Mode')

                 PP_Arousal_Mode
PP_Arousal_Mode                 
neutral                    34.17
relaxed                    32.66
stressed                   33.17 

                 HR_Arousal_Mode
HR_Arousal_Mode                 
neutral                    34.02
relaxed                    31.90
stressed                   34.07 

                 BR_Arousal_Mode
BR_Arousal_Mode                 
neutral                    33.68
relaxed                    31.93
stressed                   34.39 

                    PP_HR_Arousal_Mode
PP_HR_Arousal_Mode                    
neutral                          55.67
relaxed                          21.67
stressed                         22.66 

                    PP_BR_Arousal_Mode
PP_BR_Arousal_Mode                    
neutral                          56.55
relaxed                          21.17
stressed                         22.28 

                    HR_BR_Arousal_Mode
HR_BR_Arousal_Mode                    
neutral                    

In [8]:
# discard_neutral = False

In [9]:
start_time = time.time()

for study_combination in study_combinations_to_run:
    
    train_studies = study_combinations[study_combination]['Train']
    test_studies = study_combinations[study_combination]['Test']
    # print(train_studies, test_studies, study_combination)
    
    # if '___'.join(train_studies) in all_study_combinations:
    train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    model_metrics = {}
    final_prediction_df = pd.DataFrame()

    for arousal_signal in arousal_signals:

        train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
        # if ground_truth_method == 'ecdf': 
        # if discard_neutral: 
        #     train_df = train_df[train_df[arousal_signal+'_Arousal_Mode'] != 'neutral']

        try:
            print(decorator + '\n' + decorator)
            print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
            print(decorator + '\n' + decorator + '\n')


            # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
            model_metrics[arousal_signal] = {
                'K_Fold': copy.deepcopy(all_metrics),
                'Train': copy.deepcopy(all_metrics),
                'Test': {}
            }

            # print(train_df[arousal_signal+'_Arousal_Mode'].unique())
            model = get_trained_model(train_df, arousal_signal)

            # for i, test_study in enumerate(test_studies):
            for j in range(1, len(test_studies)+1):
                for subset in itertools.combinations(test_studies, j):

                    test_study_subset = list(subset)
                    test_study_str = '___'.join(test_study_subset)

                    if test_study_str in all_study_combinations:

                        # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
                        print('\n------------------------ Test Study: ' + test_study_str.upper() + ' ------------------------')

                        model_metrics[arousal_signal]['Test'][test_study_str.upper()] = copy.deepcopy(all_metrics)
                        # test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
                        test_df = all_studies_df.copy()[all_studies_df.Study_Name == test_study_str]

                        # if ground_truth_method == 'ecdf':
                        # if discard_neutral: 
                        #     test_df = test_df[test_df[arousal_signal+'_Arousal_Mode'] != 'neutral']
                        prediction_df = test_model(model, test_study_str.upper(), test_df, arousal_signal)

                        final_prediction_df = final_prediction_df.append(prediction_df)
                        metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics), ignore_index = True)

            if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics), ignore_index = True)

        except Exception as e:
            print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))


    # pprint.pprint(model_metrics)
    # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
    study_combination_model_metrics[study_combination] = model_metrics

    
display(metrics_df)
metrics_file_name = 'ml_classification_deadline_____' if is_deadline else 'ml_classification_____'
metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + metrics_file_name + get_date_time() + '.csv', sep=',', index=False)


end_time = time.time()

----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: PP
----------------------------------------------------
----------------------------------------------------

Running 5-fold iteration: 1

Running 5-fold iteration: 2

Running 5-fold iteration: 3

Running 5-fold iteration: 4

Running 5-fold iteration: 5


Avg. k-fold metrics: ----> 
Accuracy: 0.976089
AUC: 0.981462
F1: 0.976076
Recall: 0.976089
Precision: 0.976113
Specificity: 0



------------------------ Test Study: SIM2 ------------------------

------------------------ Test Study: TT1 ------------------------

------------------------ Test Study: OFFICE_TASKS ------------------------
----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: HR
----------------------------------------------------
----------------------------------------------------

Running 5-fold it

,Model,Arousal_Signal,Train_Study,Test_Study,Train_Accuracy,Test_Accuracy,Train_AUC,Test_AUC,Train_F1,Test_F1,Train_Recall,Test_Recall,Train_Precision,Test_Precision,Train_Specificity,Test_Specificity
0,KNN,PP,sim1,sim2,0.976089,0.82385,0.981462,0.869643,0.976076,0.807622,0.976089,0.82385,0.976113,0.867454,0.0,0.0
1,KNN,PP,sim1,tt1,0.976089,0.807229,0.981462,0.857951,0.976076,0.805591,0.976089,0.807229,0.976113,0.814814,0.0,0.0
2,KNN,PP,sim1,office_tasks,0.976089,0.649281,0.981462,0.73557,0.976076,0.632145,0.976089,0.649281,0.976113,0.686541,0.0,0.0
3,KNN,HR,sim1,sim2,0.932270,0.78575,0.949282,0.843123,0.932396,0.778852,0.932270,0.78575,0.933297,0.807726,0.0,0.0
4,KNN,HR,sim1,tt1,0.932270,0.912377,0.949282,0.934846,0.932396,0.910595,0.932270,0.912377,0.933297,0.914028,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,KNN,BR,sim1___sim2___tt1___office_tasks,None,0.778029,None,0.834408,None,0.773484,None,0.778029,None,0.803303,None,0.0,None
143,KNN,PP_HR,sim1___sim2___tt1___office_tasks,None,0.891319,None,0.903381,None,0.889777,None,0.891319,None,0.898411,None,0.0,None
144,KNN,PP_BR,sim1___sim2___tt1___office_tasks,None,0.804152,None,0.818645,None,0.797457,None,0.804152,None,0.831074,None,0.0,None
145,KNN,HR_BR,sim1___sim2___tt1___office_tasks,None,0.832717,None,0.859821,None,0.828554,None,0.832717,None,0.853090,None,0.0,None


In [10]:
# prediction_cols = ['Subject', 'Study_Name', 'PP_Arousal_Mode', 'Prediction'] + model_features

# final_prediction_df = final_prediction_df[prediction_cols]
# final_prediction_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + 'final_prediction_df_v1.csv', sep=',', index=False)

# final_prediction_df = final_prediction_df.loc[~(final_prediction_df['PP_Arousal_Mode'] == final_prediction_df['Prediction'])]
# final_prediction_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + 'final_prediction_df_v2.csv', sep=',', index=False)


In [11]:
modeling_time = end_time - start_time
print("--- Total Time %s mins (%s hours) ---" % (modeling_time//60, modeling_time//3600))


# --- Total Time 25.0 mins (0.0 hours) ---


--- Total Time 12.0 mins (0.0 hours) ---


In [12]:
# # run_models = True
# # print_all_metrics = False
# %run us_common_functions.ipynb


# if run_models:
#     for study_combination in study_combinations_to_run:
#         overleaf_final_table = get_overleaf_table(study_combination, study_combination_model_metrics[study_combination])
#         overleaf_final_tables[study_combination] = overleaf_final_table

#     with open(add_path(all_studies_data_dir, metrics_dir) + 'overleaf_final_tables_' + '_'.join(study_combinations_to_run) + '.json', 'w') as json_file:
#         json.dump(overleaf_final_tables, json_file)
    

# if not run_models:
#     with open(add_path(all_studies_data_dir, metrics_dir) + 'overleaf_final_tables_' + '_'.join(study_combinations_to_run) + '.json') as json_file:
#         overleaf_final_tables = json.load(json_file)


# for study_combination, overleaf_table in overleaf_final_tables.items():
#     print('%' + decorator + '\n% Train Studies: ' + study_combination.upper() + '\n%' + decorator + '\n')
#     print(overleaf_table + '\n\n\n\n')

In [13]:
# start_time = time.time()

# for study_combination in study_combinations_to_run:
    
#     train_studies = study_combinations[study_combination]['Train']
#     test_studies = study_combinations[study_combination]['Test']
#     # print(train_studies, test_studies, study_combination)
    
#     train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    
#     model_metrics = {}
    
    
#     for arousal_signal in arousal_signals:
        
#         train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
#         if ground_truth_method == 'ecdf': 
#             train_df = train_df[train_df[arousal_signal+'_Arousal_Mode'] != 'neutral']
        
#         try:
#             print(decorator + '\n' + decorator)
#             print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
#             print(decorator + '\n' + decorator + '\n')


#             # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
#             model_metrics[arousal_signal] = {
#                 'K_Fold': copy.deepcopy(all_metrics),
#                 'Train': copy.deepcopy(all_metrics),
#                 'Test': {}
#             }

#             # print(train_df[arousal_signal+'_Arousal_Mode'].unique())
#             model = get_trained_model(train_df, arousal_signal)

#             # for i, test_study in enumerate(test_studies):
#             for j in range(1, len(test_studies)+1):
#                 for subset in itertools.combinations(test_studies, j):
                    
#                     test_study_subset = list(subset)
#                     test_study_str = '___'.join(test_study_subset).upper()
#                     # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
#                     print('\n------------------------ Test Study: ' + test_study_str + ' ------------------------')

#                     model_metrics[arousal_signal]['Test'][test_study_str] = copy.deepcopy(all_metrics)
#                     test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
#                     if ground_truth_method == 'ecdf': 
#                         test_df = test_df[test_df[arousal_signal+'_Arousal_Mode'] != 'neutral']
#                     test_model(model, test_study_str, test_df, arousal_signal)

#                     metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics), ignore_index = True)

#             if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics), ignore_index = True)
        
#         except Exception as e:
#             print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
        
#     # pprint.pprint(model_metrics)
#     # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
#     study_combination_model_metrics[study_combination] = model_metrics
    
    
# display(metrics_df)
# metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + 'ml_' + get_date_time() + '.csv', sep=',', index=False)


# end_time = time.time()